In [1]:
#Model

In [2]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input

In [3]:
from tensorflow.keras.models import Model

In [4]:
def conv_block(inputs, num_filters):
    x=Conv2D(num_filters, 3, padding="same")(inputs)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    x=Conv2D(num_filters, 3, padding="same")(inputs)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    return x

In [5]:
def encoder_block(inputs, num_filters):
    x=conv_block(inputs, num_filters)
    p=MaxPool2D((2,2))(x)
    
    return x, p

In [6]:
def decoder_block(inputs, skip_features, num_filters):
    x=Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x=Concatenate()([x, skip_features])
    x=conv_block(x, num_filters)
    return x

In [7]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    """Bridge"""
    b1=conv_block(p4, 1024)
    
    """Decoder"""
    d1=decoder_block(b1, s4, 512)
    d2=decoder_block(d1, s3, 256)
    d3=decoder_block(d2, s2, 128)
    d4=decoder_block(d3, s1, 64)
    
    """Output"""
    outputs=Conv2D(1, (1,1), padding="same", activation="sigmoid")(d4)
    
    model=Model(inputs, outputs, name="U-Net")
    return model
if __name__== "__main__":
    input_shape= (512, 512, 3)
    model=build_unet(input_shape)
    model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 64) 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [8]:
#Data preprocessing

In [9]:
import tensorflow as tf
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split

def load_data(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*")))
    masks = sorted(glob(os.path.join(dataset_path, "masks/*")))
    
    train_x, test_x= train_test_split(images, test_size=0.2, random_state=42)
    train_y, test_y= train_test_split(masks, test_size=0.2, random_state=42)
    
    return (train_x, train_y), (test_x, test_y)

def read_image(path):
    x= cv2.imread(path, cv2.IMREAD_COLOR)
    x= cv2.resize(x, (256,256))
    x= x/255.0 #normalization stage
    x=x.astype(np.float32)  #Converts image to (256,256,3)
    return x
def read_mask(path):
    x= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x= cv2.resize(x, (256,256)) #Grayscale already puts image in range of zero and 1 so no need for normalization
    x= x.astype(np.float32)   #converts image to (256,256)
    x= np.expand_dims(x, axis=-1)    #converts image from (256,256) to (256,256,1)
    return x
    
def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path= image_path.decode()
        mask_path= mask_path.decode()
            
        x= read_image(image_path)
        y= read_mask(mask_path)
            
        return x,y
        
    image, mask= tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([256,256,3])
    mask.set_shape([256,256,1])
        
    return image, mask
    
def tf_dataset(images, masks, batch=8):
    dataset= tf.data.Dataset.from_tensor_slices((images, masks))
    dataset= dataset.shuffle(buffer_size=5000)
    dataset= dataset.map(preprocess)
    dataset= dataset.batch(batch)
    dataseta= dataset.prefetch(2)
    return dataset
        
if __name__ == "__main__":
    dataset_path="archive/people_segmentation"
    (train_x, train_y), (test_x, test_y)= load_data(dataset_path)
    
    
    print(f"Training: {len(train_x)} - Masks: {len(train_y)}")
    print(f"Training: {len(test_x)} - Masks: {len(test_y)}")
    
    train_dataset = tf_dataset(train_x, train_y, batch=12)
    #for image, mask in train_dataset:
       # print(image.shape, mask.shape)
        
    """
    10 - elements
    batch - 3
    10//3=3
    3*3=9
    3+1
    """
    batch=12
    train_steps=len(train_x)//batch
    if len(train_x) % batch!=0:
        train_steps+=1
        
    print("Train Steps: ", train_steps)

Training: 4542 - Masks: 4542
Training: 1136 - Masks: 1136
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Train Steps:  379


In [10]:
#Train

In [11]:
if __name__=="__main__":
    """Hyperparameters"""
    dataset_path="archive/people_segmentation"
    input_shape= (256,256,3)
    batch_size=8
    epochs=10
    lr=1e-4
    model_path="unet.h5"
    csv_path="data.csv"

In [12]:
"""loading the dataset"""
(train_x, train_y), (test_x, test_y)= load_data(dataset_path)
print (f"Training: {len(train_x)} - {len(train_y)}")
print (f"Testing: {len(test_x)} - {len(test_y)}")

train_dataset=tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset=tf_dataset(test_x, test_y, batch=batch_size)

Training: 4542 - 4542
Testing: 1136 - 1136


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

In [14]:
"""Model"""
model=build_unet(input_shape)
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision() 
    ]
)


In [15]:
callbacks=[
    ModelCheckpoint(model_path, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
    CSVLogger(csv_path), 
    EarlyStopping(monitor="val_loss", patience=10)
]

train_steps=len(train_x)//batch
if len(train_x) % batch!=0:
    train_steps+=1
    
valid_steps=len(test_x)//batch_size
if len(test_x)%batch_size!=0:
    valid_steps+=1

model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train for 379 steps, validate for 142 steps
Epoch 1/10
378/379 [============================>.] - ETA: 10s - loss: 0.4290 - mean_io_u: 0.3727 - recall: 0.5426 - precision: 0.6392
Epoch 00001: saving model to unet.h5
379/379 [==============================] - 4231s 11s/step - loss: 0.4289 - mean_io_u: 0.3727 - recall: 0.5427 - precision: 0.6393 - val_loss: 0.4261 - val_mean_io_u: 0.3721 - val_recall: 0.3930 - val_precision: 0.7283
Epoch 2/10
189/379 [=============>................] - ETA: 30:26 - loss: 0.3663 - mean_io_u: 0.3714 - recall: 0.6099 - precision: 0.7151WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 3790 batches). You may need to use the repeat() function when building your dataset.

Epoch 00002: saving model to unet.h5
189/379 [=============>................] - ETA: 30:27 - loss: 0.3663 - mean_io_u: 0.3714 - recall: 0.6099 - precision: 0.715

In [16]:
#Testing 

In [17]:
from tqdm import tqdm
import tensorflow as tf
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split

In [29]:
if __name__=="__main__":
    test_images=glob("images/*")
    
    model=tf.keras.models.load_model("unet.h5")
    
    for path in tqdm (test_images, total=len(test_images)):
        x=cv2.imread(path, cv2.IMREAD_COLOR)
        original_image=x
        h, w, _=x.shape
        
        x=cv2.resize(x, (256,256))
        x=x/255.0
        x=x.astype(np.float32)
        
        #To convert (256, 256, 3) into (1, 256, 256, 3)
        x=np.expand_dims(x, axis=0)
        
        pred_mask=model.predict(x)
        
        #To convert (1, 256, 256, 1) into (256, 256, 1)
        pred_mask=pred_mask[0]
        
        #To convert into (256, 256, 3)
        
        pred_mask=np.concatenate(
            [
                pred_mask, pred_mask, pred_mask
            ], axis=2
        )
        
        pred_mask= pred_mask>0.5 * 255
        pred_mask= pred_mask.astype(np.float32)
        pred_mask=cv2.resize(pred_mask, (w,h))
        
        original_image=original_image.astype(np.float32)
        
        alpha=0.6
        cv2.addWeighted(pred_mask, alpha, original_image, 1-alpha, 0, original_image )
        
        """
        
        images/1.png
        
        """
        name=path.split("/")[-1] #["images", "1.jpg"]
        cv2.imwrite(f"save_images/{name}", original_image)

100%|██████████| 1/1 [00:00<00:00,  1.75it/s]
